<a href="https://colab.research.google.com/github/ewu2023/CS589-HW4/blob/main/CS589_NeuralNets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Packages

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import json

# Define Neural Network Class

In [ ]:
# TODO: Add a debug flag to show intermediate computation steps
class NeuralNetwork():
    def __init__(self, networkShape, trainData: pd.DataFrame, classSet, weights=None, regParam=0, debugFlag=False):
        """
        Constructor for the neural network class.
        
        networkShape: A list of integers that contains the number of neurons to use in each layer

        trainData: The data set that will be used to train the model

        weights: A list of weight matrices for each hidden layer. If initialized to None, the constructor will assign random weights
        """

        """
        Process training data
        """
        self.trainData = trainData

        
        # Get a copy of the training data without the labels
        self.noLabelTrainData = trainData.loc[:, trainData.columns!='class']

        # Encode class vectors
        self.classVectors = {}
        for i in range(len(trainData)):
            # Get current row from training data
            row = trainData.iloc[i]

            # Get current class
            rowClass = row['class']

            # Create class vector (Need to format class labels as numbers)
            classVector = np.zeros(shape=(1, len(classSet)))
            classVector[rowClass - 1] = 1

            self.classVectors[i] = classVector 

        self.networkShape = networkShape

        # Set the value of the regularization parameter
        self.regParam = regParam

        # Set debug flag to show output of intermediate computations
        self.debugFlag = debugFlag

        # Instance variable storing weight matrices
        self.layers = []
        
        # Initialize layers
        for i in range(len(networkShape) - 1):
            # Get the number of neurons in the current and next layers
            numCurLayer = networkShape[i] + 1 # Account for neurons + bias term in current layer
            numNextLayer = networkShape[i + 1]

            # Initialize matrix for the current layer
            # Number of rows = number of neurons in layer i + 1
            # Number of columns = number of neurons in layer i
            layerMatrix = np.zeros(shape=(numNextLayer, numCurLayer))
            if weights:
                layerMatrix = weights[i]
            else:
                self._init_matrix(layerMatrix)
            
            # Append current layer to the list of layers
            self.layers.append(layerMatrix)
        
    def _init_matrix(self, matrix: np.ndarray):
        rows, cols = matrix.shape
        for i in range(rows):
            for j in range(cols):
                matrix[i, j] = norm.rvs()
    
    # Definition for the sigmoid function
    def sigmoid(self, x):
        return (1 / (1 + np.exp(-x)))
    
    # Compute activation vector
    def compute_activation_vector(self, weightedSums: np.ndarray):
        numRows, numCols = weightedSums.shape
        activationVector = np.zeros(shape=(numRows, numCols))

        for i in range(numRows):
            # Get weighted sum from i-th row
            x = weightedSums[i, 0]

            # Compute output of sigmoid function and place it in activation vector
            activationVector[i, 0] = self.sigmoid(x)
        
        return activationVector
    
    # Method for propagating forward one instance
    def propagate_one(self, instance):
        # Add a bias term to the instance
        instanceAsNP = instance.to_numpy()
        instanceVector = np.concatenate(([1], instanceAsNP))
        
        # Make instance vector a column vector
        instanceVector = np.atleast_2d(instanceVector).T
        
        # Iterate over each layer and compute activations for each neuron
        prevActivation = instanceVector # Keep track of the activation vector for previous layer

        # If the debug flag was set, print out the first instance vector
        if self.debugFlag:
            print(f"Value of a0:\n{prevActivation}\n")

        for i in range(len(self.layers) - 1):
            # Get current weight matrix
            curTheta = self.layers[i]

            # Compute weighted sum vector (z-matrix): Theta^{l=i-1} * a^{l=i-1}
            z = np.matmul(curTheta, prevActivation)

            # Compute activation vector of current layer
            curActivationVec = self.compute_activation_vector(z)
            
            # Add bias term to current activation vector
            curActivationVec = np.concatenate(([[1]], curActivationVec)) # Prepend 1 to vector

            # Update previous activation vector
            prevActivation = curActivationVec

            # Print results of computation at this step if debug flag is on
            if self.debugFlag:
                print(f"Value of z{i + 1}:\n{z}")
                print(f"Value of a{i + 1}:\n{curActivationVec}\n")
        
        # Compute activation at the final layer
        lastTheta = self.layers[len(self.layers) - 1]
        lastZMat = np.matmul(lastTheta, prevActivation)
        outputVector = self.compute_activation_vector(lastZMat)

        # If the debug flag was set, print results of this final computation
        if self.debugFlag:
            print(f"Value of z{len(self.layers)}:\n{lastZMat}")
            print(f"Value of a{len(self.layers)}:\n{outputVector}\n")

        # Return as a vector in the event there are multiple outputs
        return outputVector

    # Compute error for an individual output of the neural network
    def compute_one_instance_err(self, expVal, predVal):
        return -expVal * np.log(predVal) - (1 - expVal) * np.log(1 - predVal)

    # Helper method for computing regularized error
    def compute_error(self):
        # Keep track of total error across all training instances
        totalErr = 0

        # Iterate over all training instances
        for i in range(len(self.noLabelTrainData)):
            # Get current instance and perform forward propagation on it
            curInstance = self.noLabelTrainData.iloc[i]
            predVector = self.propagate_one(curInstance) # Will be a vector

            # Get expected value for current instance
            expVal = (self.trainData.iloc[i]['class'])

            # Build expected values vector
            # Vector takes the form of [0, 0, ..., expVal, 0, ..., 0]
            # Number of output neurons = number of classes
            numNeurons = self.networkShape[len(self.networkShape) - 1]
            expVector = np.zeros(shape=(numNeurons, 1))
            
            # How to determine where to place 0s and where to place the expected value?
            expVector[0, 0] = expVal

            # Compute error vector
            vectorizedErrorFunc = np.vectorize(self.compute_one_instance_err)
            errVector = vectorizedErrorFunc(expVector, predVector)

            # Sum all elements of error vector, then add it to total error
            totalErr += np.sum(errVector)
        
        # Compute average error
        avgErr = totalErr / len(self.trainData)

        """ Compute the squared sum of all weights in the network """
        weightSqSum = 0
        for weightMatrix in self.layers:
            # Square all of the weights
            squaredMatrix = np.multiply(weightMatrix, weightMatrix)

            # Add all columns, then add each column's total to get the total sum for this matrix
            colSums = np.sum(squaredMatrix, axis=0)
            matrixTotal = np.sum(colSums)

            # Add matrix total to sum of the weights squared
            weightSqSum += matrixTotal
        
        # Regularize error
        weightSqSum *= (self.regParam / (2 * len(self.trainData)))

        # Return error + regularization term
        return avgErr + weightSqSum

Test Neural Net

In [ ]:
d = {
    'x': [0.13000, 0.42000], 
    'class': [0.90000, 0.23000]
}

df = pd.DataFrame(data=d)
df_noLabels = df.loc[:, df.columns!='class']

networkShape = [1, 2, 1]
weights = [
    np.array(
        [[0.40000, 0.10000],
         [0.30000, 0.20000]],
    ),

    np.array([[0.70000, 0.50000, 0.60000]])
]

network = NeuralNetwork(networkShape, trainData=df, weights=weights, debugFlag=True)
print(network.compute_error())

Value of a0:
[[1.  ]
 [0.13]]

Value of z1:
[[0.413]
 [0.326]]
Value of a1:
[[1.       ]
 [0.601807 ]
 [0.5807858]]

Value of z2:
[[1.34937498]]
Value of a2:
[[0.79402743]]

Value of a0:
[[1.  ]
 [0.42]]

Value of z1:
[[0.442]
 [0.384]]
Value of a1:
[[1.        ]
 [0.60873549]
 [0.59483749]]

Value of z2:
[[1.36127024]]
Value of a2:
[[0.79596607]]

0.8209757904998143


# Test Code

In [ ]:
# target_dir = "/content/drive/My Drive/"
# filename = "blarg.json"
# fileRoute = f"{target_dir}/{filename}"

# with open(fileRoute, 'w') as outfile:
#     some_data = {
#         "a1": [[1, 2], [3, 4]],
#         "b1": [[5, 6], [7, 8]]
#     }

#     outfile.write(json.dumps(some_data))

# Compute error for an individual output of the neural network
def compute_one_instance_err(expVal, predVal):
    return -expVal * np.log(predVal) - (1 - expVal) * np.log(1 - predVal)

def multiply(a, b):
    return a * b


vectorFunction = np.vectorize(multiply)
a = np.array([
    [1, 2],
    [3, 4]
])

colSum = np.sum(a, axis=0)
rowSum = np.sum(a, axis=1)

print(colSum)
print(rowSum)
print(f"Sum of all elements in matrix: {np.sum(colSum)}")

b = np.array([
    [3],
    [7]
])

print(f"b-vector elements squared:\n{np.multiply(b, b)}")

[4 6]
[3 7]
Sum of all elements in matrix: 10
b-vector elements squared:
[[ 9]
 [49]]
